# Data 3 - Power
- 從 2016-09-27 到 2017-09-01 的電力資料
- 北中南東各有兩個 columns ， 分別代表供給量及使用量 (Supply & Usage)
- 約每小時一筆資料

### Step 1: Transaction Definition

#### - Find the associate rules between "Date", "Time", "NorthSupply",  "NorthUsage",   "SouthSupply",  "SouthUsage",  "EastSupply",  "EastUsage", "CenterSupply", and "CenterUsage".

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import time
from sklearn.cluster import KMeans
from pylab import rcParams
%matplotlib inline

power = pd.read_csv("power.csv")
power.head()

,Date,Time,NorthSupply,NorthUsage,CenterSupply,CenterUsage,SouthSupply,SouthUsage,EastSupply,EastUsage
0,2016-09-27,12:10,648.4,841.3,733.0,564.4,839.6,799.9,14.0,29.3
1,2016-09-27,13:10,601.3,826.4,725.4,535.9,827.3,777.9,14.0,27.6
2,2016-09-27,14:00,500.9,789.5,759.0,506.7,821.5,771.3,13.9,27.9
3,2016-09-27,15:10,516.1,773.6,717.1,450.8,744.2,737.3,13.9,29.5
4,2016-09-27,16:00,530.2,778.0,691.6,432.2,746.0,735.3,8.7,31.0


In [2]:
power.describe()

,NorthSupply,NorthUsage,CenterSupply,CenterUsage,SouthSupply,SouthUsage,EastSupply,EastUsage
count,6060.000000,6060.000000,6060.000000,6060.000000,6060.000000,6060.000000,6060.000000,6060.000000
mean,871.774059,970.082541,760.360231,708.192706,977.277937,901.751617,8.068531,37.453020
std,169.168890,206.794644,160.876822,103.979344,138.950242,115.564858,3.983699,6.158678
min,429.200000,574.600000,329.000000,420.700000,670.200000,604.500000,0.200000,23.200000
25%,750.500000,807.450000,638.475000,629.600000,869.975000,815.875000,4.600000,32.700000
50%,863.300000,950.900000,766.700000,702.150000,965.300000,897.700000,7.500000,37.100000
75%,998.500000,1110.400000,891.200000,788.225000,1069.225000,995.125000,11.200000,41.600000
max,1237.000000,1491.700000,1166.000000,947.000000,1365.000000,1175.900000,17.600000,52.400000
